In [43]:
import pandas as pd
from functions import get_feature_columns
from functions import X_log
from functions import X_standard
from functions import xy
from functions import return_by_global_mean

In [44]:
df = pd.read_csv("Cleaned_Data_0506.csv")

In [45]:
df

,care_team,age,occupation,intake_no_of_hh,income_assessment_salary,income_assessment_cpf_payout,income_assessment_assistance_from_other_agencies,income_assessment_assistance_from_relatives_friends,income_assessment_insurance_payout,income_assessment_rental_income,...,primary_7_12,secondary_13_17,tertiary_18_21,adult_22_64,elderly_65_and_above,income_total_cal,expenditure_total_cal,difference_cal,amount_total,points
0,central 1 1,13.0,student,3.0,0.0,0.00,500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,500.00,1601.9,-1101.90,900.00,0
1,central 1 1,57.0,unemployed,2.0,1500.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1500.00,1084.0,416.00,1144.90,0
2,central 1 1,32.0,unemployed,4.0,3380.0,0.00,0.0,500.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3880.00,3494.0,386.00,243.00,0
3,central 1 1,69.0,NaN,2.0,525.0,0.00,200.0,400.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1125.00,1114.0,11.00,243.00,0
4,central 1 1,70.0,NaN,1.0,134.0,0.00,90.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,474.00,700.0,-226.00,200.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,south 1 1,55.0,unemployed,4.0,0.0,0.00,1780.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2.0,0.0,1780.00,1730.0,50.00,717.20,1
941,east 1 1,55.0,employed,1.0,0.0,0.00,600.0,150.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,750.00,2285.0,-1535.00,396.80,2
942,south 2 2,62.0,unemployed,3.0,0.0,0.00,1080.0,0.0,0.0,500.0,...,0.0,0.0,0.0,0.0,0.0,1580.00,1530.0,50.00,126.00,0
943,east 2 2,66.0,employed,0.0,2800.0,876.95,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,3.0,0.0,3676.95,2416.5,1260.45,1213.15,2


In [48]:
model = input('type of model ?')
method = input('onehot or label ?')
if_log = input('log, yes or no ?')
if_standard = input('standard, yes or no ?')
if_pca = input('pca, yes or no ?')
if_predict = input('if liyan, plz input no; if caoyang, plz input yes')

df = get_feature_columns(df, method)

if if_log == 'yes':
    df = X_log(df)
else:
    pass

if if_standard == 'yes':
    df = X_standard (df, method, if_log)
else:
    pass

if if_predict == 'no':
    X,y = xy(df_1, method, if_log, if_pca, if_predict)
    print(X,y)
else:
    X = xy(df_1, method, if_log, if_pca, if_predict)
    print(X)


[[ 4.83309669 -0.62959076 -4.14692562 ... -0.94258126  0.03340713
  -0.7520051 ]
 [-2.41565894 -5.02093343  0.50597241 ...  0.25501117  0.82796317
  -0.35496337]
 [-6.97106892 -1.35647033 -1.42227741 ...  0.41877026  0.46168459
   0.72096122]
 ...
 [ 2.81634395  6.21989596  2.67233985 ... -0.67774297 -0.38150023
   1.38091003]
 [-9.17602696  2.61210063  2.99708906 ... -0.1773548  -0.23630062
  -0.0254883 ]
 [ 5.73507614  0.30873822 -4.37764407 ... -0.2533449   0.59442653
  -0.71828699]] 0      157.693223
1      402.593223
2     -499.306777
3     -499.306777
4     -542.306777
          ...    
940    220.287500
941   -462.718571
942   -482.144217
943    231.870185
944   -346.912500
Name: amount_adjusted_label, Length: 945, dtype: float64


/opt/anaconda3/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)


In [49]:
global_mean = return_by_global_mean(df)

In [9]:
# randomforest

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score

# 拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 网格搜索的参数范围（可按需调整）
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None]
}

# 初始化 Random Forest 模型
rf = RandomForestRegressor(random_state=42)

# 网格搜索 + 交叉验证
grid = GridSearchCV(rf, param_grid, cv=5, scoring='r2')
grid.fit(X_train, y_train)

# 最优模型预测（预测的是偏移量）
predicted_adjusted = grid.predict(X_test)

# 还原预测值（用 team 均值还原为原始 amount_total）
final_pred = predicted_adjusted + global_mean

# 真实值（从原始 df 中拿出 y 的原始值）
y_true = df.loc[y_test.index, "amount_total"]

# 模型效果评估
r2 = r2_score(y_true, final_pred)
print("R² 分数:", round(r2, 4))
print("真实值:", y_true.values)
print("预测值:", final_pred.round(2))


R² 分数: 0.3896
真实值: [1663.15  243.   1050.      0.   1120.    430.    910.    900.54 1020.4
 1663.15 1328.2  1023.35 1778.2   200.   1520.9     0.    575.5   300.
  185.5   131.2     0.    319.37 2600.    266.6  1000.6  1170.     88.
  800.   1255.66  117.    800.   1549.6    91.    836.8   630.   1420.
  150.    300.   1520.9  1255.66 1080.      0.   1195.15 1213.15  200.
 1213.15  929.75  580.    660.    150.    200.    320.   1680.     75.15
  720.    949.17 1050.    144.   1637.8   405.      0.    150.   1005.96
 1328.2  1298.17 1851.1   150.    790.    701.33  150.    911.05 1047.4
 1507.8   246.89  364.8    64.5   100.   1778.2   812.95  200.      0.
  318.5  1036.8  1298.17 1637.8   286.    323.35  192.   1804.85  850.
  792.6    97.2   600.4  1398.4  1187.8   200.    200.    717.2   590.1
  617.94  200.    117.84 1517.26  260.    671.94  800.      0.    891.25
 1637.8  1430.     49.2  1260.   1328.2   885.4   200.05 1040.   1663.15
  150.   1893.76  847.6   151.9  1539.06  105.0

In [50]:
# KNN

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# 拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

param_grid = {'n_neighbors': list(range(1, 31))}
knn = KNeighborsRegressor()
grid = GridSearchCV(knn, param_grid, cv=5, scoring='r2')
grid.fit(X_train, y_train)

final_pred = predicted_adjusted + global_mean

y_true = df.loc[y_test.index, "amount_total"]

r2 = r2_score(y_true, final_pred)
print("R² 分数:", round(r2, 4))
print("真实值:", y_true.values)
print("预测值:", final_pred.round(2))


R² 分数: 0.3896
真实值: [1663.15  243.   1050.      0.   1120.    430.    910.    900.54 1020.4
 1663.15 1328.2  1023.35 1778.2   200.   1520.9     0.    575.5   300.
  185.5   131.2     0.    319.37 2600.    266.6  1000.6  1170.     88.
  800.   1255.66  117.    800.   1549.6    91.    836.8   630.   1420.
  150.    300.   1520.9  1255.66 1080.      0.   1195.15 1213.15  200.
 1213.15  929.75  580.    660.    150.    200.    320.   1680.     75.15
  720.    949.17 1050.    144.   1637.8   405.      0.    150.   1005.96
 1328.2  1298.17 1851.1   150.    790.    701.33  150.    911.05 1047.4
 1507.8   246.89  364.8    64.5   100.   1778.2   812.95  200.      0.
  318.5  1036.8  1298.17 1637.8   286.    323.35  192.   1804.85  850.
  792.6    97.2   600.4  1398.4  1187.8   200.    200.    717.2   590.1
  617.94  200.    117.84 1517.26  260.    671.94  800.      0.    891.25
 1637.8  1430.     49.2  1260.   1328.2   885.4   200.05 1040.   1663.15
  150.   1893.76  847.6   151.9  1539.06  105.0